Calculate ensemble of sea level projections

In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys
sys.path.append('../src')

from alldata import AllData
from ensemble import EnsembleMember,FullEnsemble

np.set_printoptions(precision=2)
%matplotlib notebook
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

Read data

In [ ]:

ad = AllData()
ad.usefanom = True
ad.gather(update=False)
#Select subsection of models for rapid calculation
#ad.ds = ad.ds.isel(ism=slice(2,4),esm=slice(0,2))

fe = FullEnsemble(ad)
fe.gather(force_update=False)

In [ ]:

ad = AllData()
ad.usefanom = True
ad.gather(update=False)
#Select subsection of models for rapid calculation
#ad.ds = ad.ds.isel(ism=slice(2,4),esm=slice(0,2))

fe = FullEnsemble(ad)
fe.gather(force_update=True)

Figure settings

In [ ]:
plt.style.use('../src/style_paper')
mpl.rc('figure.subplot',left=.15,bottom=.2)

alpha = .3 #Transparency of uncertainty range
perc = [17,83] #Percentiles of uncertainty range

Make and save figure

In [ ]:
fig,ax = plt.subplots(2,3,sharex=True,sharey='row',figsize=(7,5))

for s,ss in enumerate(ad.ssp):
    for a in [0,1,2]:
        for t in range(len(fe.time)):
            fe.slr[a,s,:,:,t] = np.where(np.isnan(fe.gamma[1,:,:]),np.nan,fe.slr[a,s,:,:,t])
            fe.slr[a,s,:,:,t] = np.where(fe.gamma[0,:,:]<0,np.nan,fe.slr[a,s,:,:,t])
    
    
    #Plot no feedback
    for a in [0,1]:
        ax[a,s].plot(fe.time,100*np.nanmedian(fe.slr[0,s,:,:,:],axis=(0,1)),c='.5',lw=2,label='No feedback')
        ax[a,s].fill_between(fe.time,100*np.nanpercentile(fe.slr[0,s,:,:,:],perc[0],axis=(0,1)),100*np.nanpercentile(fe.slr[0,s,:,:,:],perc[1],axis=(0,1)),color='.5',alpha=alpha)#,label=perc)

    #Plot with feedback
    ax[0,s].plot(fe.time,100*np.nanmedian(fe.slr[1,s,:,:,:],axis=(0,1)),'--',c=ad.scol[ss],lw=2,label='Feedback')
    ax[0,s].fill_between(fe.time,100*np.nanpercentile(fe.slr[1,s,:,:,:],perc[0],axis=(0,1)),100*np.nanpercentile(fe.slr[1,s,:,:,:],perc[1],axis=(0,1)),color=ad.scol[ss],alpha=alpha)

    #Plot with feedback
    ax[1,s].plot(fe.time,100*np.nanmedian(fe.slr[2,s,:,:,:],axis=(0,1)),c=ad.scol[ss],lw=2,label='Feedback')
    ax[1,s].fill_between(fe.time,100*np.nanpercentile(fe.slr[2,s,:,:,:],perc[0],axis=(0,1)),100*np.nanpercentile(fe.slr[2,s,:,:,:],perc[1],axis=(0,1)),color=ad.scol[ss],alpha=alpha)
    
    ax[0,s].set_title(f'SSP-{ss}')
    
    print(ss,'SLR_amp','br',np.nanmedian(fe.slr[1,s,:,:,:],axis=(0,1))[-1]/(np.nanmedian(fe.slr[0,s,:,:,:],axis=(0,1))[-1]))
    print(ss,'SLR_amp','ar',np.nanmedian(fe.slr[2,s,:,:,:],axis=(0,1))[-1]/(np.nanmedian(fe.slr[0,s,:,:,:],axis=(0,1))[-1]))
    
ax[0,0].legend(loc='upper left')
fig.supylabel('Cumulative sea-level rise [cm]')
ax[0,0].set_ylabel('Before recalibration',fontsize=9)
ax[1,0].set_ylabel('After recalibration',fontsize=9)
ax[1,1].set_xlabel('Time [years]')

plt.savefig(f'../draftfigs/full_ensemble_{fe.year0}.png',dpi=450,facecolor='w',transparent=False)
plt.show()